### 네이버 시가총액 (코스피500개, 코스닥300개)

In [ ]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

def stock_data(columns, sosok):
    data = list()
    data = data + [sosok]
    code = list()    
    #fn_link = list()
    for column in columns:        
        data = data + [column.text.strip()]            
        a = column.find("a",{"class":"tltle"})

        if a:
            code = [a.attrs["href"][20:]]
            #code = ["https://finance.naver.com" + a.attrs["href"]] 
            code_str = str(code)[4:10]
            #fn_link = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A"+code_str+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701"
            fn_link = "https://comp.fnguide.com/SVO/WooriRenewal/Snapshot.asp?pGB=12&gicode=A"+code_str
            #print(fn_link)
            data = data + code + [fn_link]

    return data[0:-1]



filename = "251205_sise_code.csv"

f = open(filename, "w", encoding="euc-kr", newline="")
writer = csv.writer(f)

title = "구분,No,종목명,종목코드,fn링크,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE".split(",")
print(type(title))
writer.writerow(title)

for sosok in range(0, 2):
    if sosok==0:
        last_page = 11
    else:
        last_page = 7    

    url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=" + str(sosok) + "&page="

    for page in range(1, last_page):
        res =requests.get(url + str(page))
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "lxml")

        data_rows = soup.find("table", attrs={"class":"type_2"}).find("tbody").find_all("tr")
        for row in data_rows:
            columns = row.find_all("td")
            #print(columns)
            #print("*****")
            if len(columns) <= 1:
                continue        
            #data = [column.get_text().strip() for column in columns]
            #print(data)
            #writer.writerow(data)
            
            data = stock_data(columns, sosok)
            #print(type(data))
            #print(data)

            writer.writerow(data)

print('save')
f.close()


In [ ]:
import csv
import pandas as pd

df = pd.read_csv('251205_sise_code.csv', encoding="euc-kr")

df2 = pd.DataFrame(df)
df3 = df2.sort_values(by=['등락률','전일비','현재가'], ascending=False)

display(df3)   

# df['등락률2']=df['등락률'].str.replace('%','').astype(float)
# df['상장주식수2']=df['상장주식수'].str.replace(',','').astype(int)
# df['시가총액2']=df['시가총액'].str.replace(',','').astype(int)
# df['종목코드2']=df['종목코드'].str.replace(',','').astype(str)

# df2=df.loc[df['등락률2'] > 3,:]
# #df2.loc[df2['상장주식수2'] > 50000, :]
# df2.loc[df2['시가총액2'] > 8000, :]

#my = pd.DataFrame()
#for code in codes:
#    my = df.loc[df['종목코드2']==code,'종목명':'ROE']
#display(my)

# my = df.loc[df['종목코드2'].str.contains(codes)]
# display(my[['종목명','종목코드','현재가','전일비','등락률']])